# Take 2: Adaboost with Weak Classifiers
None of that bagging classifier bullshit

In [1]:
# Import guys
import pandas as pd
import numpy as np
from scipy import sparse
# from AdaBoostClassifier import AdaBoostClassifier
#%run AdaBoostWeak.py
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


### Import data

In [2]:
# column names
with open('data/column_names.txt', 'r') as f:
    column_names = [line.strip() for line in f]

sparse_dat = sparse.load_npz("data/sparse_df.npz")

# Extract labels from the first column
labels = sparse_dat[:, 0]

# Create a list of column indices to keep
to_keep = list(set(range(sparse_dat.shape[1])) - set([0]))

# Extract the design matrix
X = sparse_dat[:, to_keep]

print(f'labels: \n{labels}')

labels: 
  (2, 0)	1
  (5, 0)	1
  (8, 0)	1
  (9, 0)	1
  (11, 0)	1
  (12, 0)	1
  (15, 0)	1
  (19, 0)	1
  (34, 0)	1
  (42, 0)	1
  (54, 0)	1
  (56, 0)	1
  (65, 0)	1
  (67, 0)	1
  (68, 0)	1
  (93, 0)	1
  (95, 0)	1
  (114, 0)	1
  (117, 0)	1
  (120, 0)	1
  (121, 0)	1
  (123, 0)	1
  (134, 0)	1
  (135, 0)	1
  (139, 0)	1
  :	:
  (15952, 0)	1
  (15953, 0)	1
  (15954, 0)	1
  (15955, 0)	1
  (15956, 0)	1
  (15957, 0)	1
  (15958, 0)	1
  (15959, 0)	1
  (15960, 0)	1
  (15961, 0)	1
  (15962, 0)	1
  (15963, 0)	1
  (15964, 0)	1
  (15965, 0)	1
  (15966, 0)	1
  (15967, 0)	1
  (15968, 0)	1
  (15969, 0)	1
  (15970, 0)	1
  (15971, 0)	1
  (15972, 0)	1
  (15973, 0)	1
  (15974, 0)	1
  (15975, 0)	1
  (15976, 0)	1


In [3]:
print(f'Design matrix: \n{X}')

Design matrix: 
  (17173, 0)	1
  (17240, 0)	2
  (17164, 1)	1
  (17363, 1)	1
  (17448, 1)	1
  (17910, 1)	1
  (17914, 1)	1
  (17933, 1)	2
  (15801, 2)	1
  (15867, 2)	1
  (16217, 2)	1
  (17173, 2)	3
  (17189, 2)	1
  (17386, 2)	1
  (17765, 2)	4
  (17933, 3)	2
  (17325, 4)	1
  (17366, 4)	1
  (16271, 5)	1
  (16837, 5)	1
  (17933, 6)	2
  (17933, 7)	2
  (17173, 8)	2
  (17933, 8)	1
  (15801, 9)	1
  :	:
  (8386, 56199)	2
  (8669, 56200)	1
  (10316, 56200)	1
  (8805, 56201)	1
  (9218, 56201)	1
  (9565, 56202)	1
  (9814, 56202)	1
  (6232, 56203)	1
  (9353, 56203)	1
  (5661, 56204)	9
  (6860, 56204)	1
  (8698, 56204)	1
  (125, 56205)	1
  (1228, 56205)	1
  (4422, 56205)	1
  (5648, 56206)	1
  (7464, 56206)	1
  (5648, 56207)	2
  (6025, 56208)	2
  (6025, 56209)	2
  (6151, 56210)	2
  (8669, 56211)	1
  (10316, 56211)	1
  (6340, 56212)	1
  (8696, 56212)	1


### Train and Test split
To do: consider stratifying by type

In [30]:
# To do - stratify the split 
n_samples = labels.shape[0]
# Use train_test_split.
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, shuffle=False)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.dtype)
print("proportion of spam in training data:", (y_train == 1).sum().item() / y_train.shape[0])
print("proportion of spam in testing data:", (y_test == 1).sum().item() / y_test.shape[0])

(14524, 56213)
(14524, 1)
(3632, 56213)
(3632, 1)
int64
proportion of spam in training data: 0.39568989259157256
proportion of spam in testing data: 0.11921806167400881


In [28]:
def errors(y, y_pred):
        """
        Calculate the proportion of type 2 errors - when the true label is 1 - spam, and the predicted label is 0 - ham

        Args:
        y: true labels
        y_pred: predicted labels
        """
        n = y.shape[0]
        type2errors = ((y == 1) & (y_pred == 0)).sum().item()
        type1errors = ((y == 0) & (y_pred == 1)).sum().item()
        correct = (y_pred == y).sum().item()
        return type2errors, type1errors, correct

### Train/Test models

2 models:
1. Without penalty
2. With penalty

In [40]:
%run AdaBoostWeak.py

aboost1 = AdaBoostWeak(type2penalty = False, rounds = 200, maxDTdepth = 5)
aboost1.fit(X = X_train, y = y_train.toarray().ravel())



In [41]:
predictions = aboost1.predict(X_test)
type2, type1, correct = errors(y_test.toarray().ravel(), predictions)
print("Model 1 Test Accuracy: ", correct/len(predictions))
print(f'unique predictions - should be 0 and 1: {np.unique(predictions)}')
print(f'type 2 errors: {type2} \n type 1 errors: {type1}')
print(aboost1.alphas)
print(predictions)

Model 1 Test Accuracy:  0.6222466960352423
unique predictions - should be 0 and 1: [0 1]
type 2 errors: 8 
 type 1 errors: 1364
[1.738078422686433, 1.1578772472886218, 0.670030453820361, 0.7881697439875254, 0.4018609691686748, 0.7307869745209322, 0.6056352062561795, 0.5039509392007954, 0.4566812695620465, 0.44703549698409745, 0.42121387903947993, 0.4364816390206816, 0.4434868817547057, 0.34417171919447914, 0.37299717539730065, 0.35279121445788303, 0.42834644827723467, 0.3362065793329317, 0.3271012928109181, 0.2947063867289413, 0.2601273589714025, 0.2329995436063459, 0.2706815519172123, 0.29078431848943437, 0.23768522419635785, 0.17803592282954409, 0.2175952200101182, 0.23261007216875126, 0.3303683941887554, 0.27645168427950634, 0.22099460959922654, 0.23421590975545464, 0.21424062167925612, 0.20469388591430593, 0.21672864514179604, 0.2109742183013699, 0.32533254680201606, 0.29097497907465014, 0.21369171457340785, 0.2566234075229289, 0.22751208339947077, 0.19908104628957038, 0.2647264306

In [42]:
predictions = aboost1.predict(X_train)
type2, type1, correct = errors(y_train.toarray().ravel(), predictions)
print("Model 1 (no penalty) Training set Accuracy: ", correct/len(predictions))
print(f'unique predictions - should be 0 and 1: {np.unique(predictions)}')
print(f'type 2 errors: {type2} \n type 1 errors: {type1}')

Model 1 (no penalty) Training set Accuracy:  0.981410079867805
unique predictions - should be 0 and 1: [0 1]
type 2 errors: 143 
 type 1 errors: 127


### Model tuning/Hyperparameter search

Sigh. So much overfitting

In [49]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score


clf = DecisionTreeClassifier(max_depth=9)
cvs = cross_val_score(clf, X_train,predictions,cv=4)
print("cross val scores ",cvs)


cross val scores  [0.81988433 0.9600661  0.90994216 0.85981823]


In [50]:

from sklearn.model_selection import GridSearchCV

clf = DecisionTreeClassifier()
param_grid = [
    {'type2penalty': [False] ,'rounds': [200, 300, 400, 500], 'maxDTdepth': [5,6,7,8,9,10]},
    {'type2penalty': [True] ,'rounds': [200, 300, 400, 500], 'maxDTdepth': [5,6,7,8,9,10]}
    ]

print(type(y_train))
print(y_train.shape)

y_train_dense = y_train.toarray().ravel()

print(y_train.shape)

grid_search = GridSearchCV(clf, param_grid, cv=4)
grid_search.fit(X_train, y_train_dense)

print(grid_search.best_params_)

<class 'scipy.sparse.csr.csr_matrix'>
(14524, 1)
(14524, 1)


ValueError: Invalid parameter maxDTdepth for estimator DecisionTreeClassifier(). Check the list of available parameters with `estimator.get_params().keys()`.

In [47]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score
param_grid = [
    {'type2penalty': [False] ,'rounds': [200, 300, 400, 500], 'maxDTdepth': [5,6,7,8,9,10]},
    {'type2penalty': [True] ,'rounds': [200, 300, 400, 500], 'maxDTdepth': [5,6,7,8,9,10]}
    ]


model = AdaBoostWeak(type2penalty=False, maxDTdepth=5,rounds=100)
scorer = make_scorer(accuracy_score)
grid_search=GridSearchCV(estimator=model, param_grid=param_grid, scoring=scorer, cv=5)
grid_search.fit(X_train, y_train.toarray().ravel())

print("best params:", grid_search.best_params_)
print("best cross val score: {:.2f}".format(grid_search.best_score_))

TypeError: Cannot clone object '<__main__.AdaBoostWeak object at 0x0000023DB9A40E20>' (type <class '__main__.AdaBoostWeak'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.